In [1]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [21]:

from detectron2.structures import BoxMode
import json
import cv2
import numpy as np

def get_warehouse_box(csv_file, img_dir):
    #json_file = os.path.join(img_dir, "via_region_data.json")
    df= pd.read_csv(csv_file)
    df['filename']= df['filename'].map(lambda x: img_dir + x)

    
    classes= ['pallet jacks', 'Rolling_Ladder', 'wire_mesh', 'bulk_box', 'totes',
       'dump_hopper', 'bin', 'bulkBox', 'yard_ramp', 'dump_hoppers']

    df['class_int']= df['class'].map(lambda x: classes.index(x))
    print(df.head())
    print(df['filename'][0])

    # with open(json_file) as f:
    #     imgs_anns = json.load(f)

    dataset_dicts = []
    for filename in df['filename'].unique().tolist():
        record = {}
        #print('initial....', filename)
        filename = os.path.join(img_dir, filename)
        #print('later.....', filename)
        try:
            height, width = cv2.imread(filename).shape[:2]

            record["file_name"] = filename
            #record["image_id"] = idx
            record["height"] = height
            record["width"] = width

            #annos = v["regions"]
            objs = []
            for index, row in df[df['filename']==filename].iterrows():
                # assert not anno["region_attributes"]
                # anno = anno["shape_attributes"]
                # px = anno["all_points_x"]
                # py = anno["all_points_y"]
                # poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
                # poly = [p for x in poly for p in x]

                obj = {
                    "bbox": [row['xmin'], row['ymin'], row['xmax'], row['ymax']],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "category_id": row['class_int'],
                    "iscrowd":0
                }
                objs.append(obj)
        except:
            print(filename)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts



In [8]:
from detectron2.data import DatasetCatalog, MetadataCatalog

classes= ['pallet jacks', 'Rolling_Ladder', 'wire_mesh', 'bulk_box', 'totes',
       'dump_hopper', 'bin', 'bulkBox', 'yard_ramp', 'dump_hoppers']

for d in ["train", "test"]:
    DatasetCatalog.register("experiment/" + d, lambda d=d: get_warehouse_box("/home/jay/warehouse/resized/experiment/" + d+ "_labels.csv", "/home/jay/warehouse/resized/experiment/"+ d +"/"))
    MetadataCatalog.get("experiment/" + d).set(thing_classes=classes)
warehouse_metadata = MetadataCatalog.get("experiment/train")

In [7]:
data= pd.read_csv('/home/jay/warehouse/resized/images/train_labels.csv')
data['class'].unique()

array(['pallet jacks', 'Rolling_Ladder', 'wire_mesh', 'bulk_box', 'totes',
       'dump_hopper', 'bin', 'bulkBox', 'yard_ramp', 'dump_hoppers'],
      dtype=object)

In [22]:
from detectron2.utils.visualizer import Visualizer
import pandas as pd

dataset_dicts= get_warehouse_box('/home/jay/warehouse/resized/images/train_labels.csv', '/home/jay/warehouse/resized/images/train/')

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=warehouse_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("images", out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

                                            filename  width  height  \
0  /home/jay/warehouse/resized/images/train/Palle...    640     640   
1  /home/jay/warehouse/resized/images/train/Palle...    640     640   
2  /home/jay/warehouse/resized/images/train/Palle...    640     640   
3  /home/jay/warehouse/resized/images/train/Palle...    640     640   
4  /home/jay/warehouse/resized/images/train/Palle...    640     640   

          class  xmin  ymin  xmax  ymax  class_int  
0  pallet jacks    70   167   169   633          0  
1  pallet jacks    88   266   411   545          0  
2  pallet jacks    26    29   614   606          0  
3  pallet jacks    15   143   620   584          0  
4  pallet jacks    82    28   394   567          0  
/home/jay/warehouse/resized/images/train/PalletJack_0145.jpg
/home/jay/warehouse/resized/images/train/PalletJack_0201.jpg
/home/jay/warehouse/resized/images/train/PalletJack_0200.jpg
/home/jay/warehouse/resized/images/train/PalletJack_0202.jpg


In [24]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import itertools
import pandas as pd


cfg= get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("/home/jay/warehouse/resized/experiment/train",)
#cfg.DATASETS.TEST = ()

cfg.DATALOADER.NUM_WORKERS = 2

cfg.SOLVER.IMS_PER_BATCH = 2
#cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
#cfg.SOLVER.MAX_ITER = 5000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8  # only has one class (ballon)
cfg.MODEL.DEVICE='cpu'

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.DATASETS.TEST= ('/home/jay/warehouse/resized/experiment/test',)

cfg.MODEL.WEIGHTS = os.path.join("path ", "where model is saved")
predictor = DefaultPredictor(cfg)

In [25]:
img= cv2.imread('/home/jay/warehouse/resized/images/test/bulkBox_00131.jpg')
output= predictor(img)
visualizer= Visualizer(img[:, :, ::-1], metadata= warehouse_metadata, scale= 0.5)
out= visualizer.draw_instance_predictions(output['instances'].to('cpu'))
#xyz= visualizer.draw_dataset_dict()
cv2.imshow('frame', out.get_image()[:, :, ::-1])
cv2.waitKey(0)

cv2.destroyAllWindows()

/home/jay/anaconda3/lib/python3.7/site-packages/detectron2/layers/wrappers.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


In [ ]:

frames1= []



cap= cv2.VideoCapture('path of the video')


while True:
    ret, frame= cap.read()
    
    output= predictor(frame)
    objs= output['instances'].get('pred_classes')


    visualizer = Visualizer(frame[:, :, ::-1], metadata=warehouse_metadata, scale=0.5)

    out = visualizer.draw_instance_predictions(output['instances'].to('cpu'))

    cv2.imshow("frame_", out.get_image()[:, :, ::-1])
    color= cv2.cvtColor(out.get_image()[:,:,::-1], cv2.COLOR_RGB2BGR)
    
    
    frames1.append(color)
    
    #clip= ImageSequenceClip(out.get_image()[:,:,::-1], fps= 20)
    #clip.write_videofile(output_vid)



    if cv2.waitKey(1) & 0xff==ord('q'):
        break

        

        
cap.release()
cv2.destroyAllWindows()

In [1]:
import tqdm
import moviepy
from moviepy.editor import *
from PIL import Image

In [9]:
output_vid= os.path.join('/home/jay/Downloads/sat_tained', 'satellite_images.mp4')

        
clip= ImageSequenceClip(frames, fps= 0.5)
clip.write_videofile(output_vid)

Moviepy - Building video /home/jay/Downloads/sat_tained/satellite_images.mp4.
Moviepy - Writing video /home/jay/Downloads/sat_tained/satellite_images.mp4



Moviepy - Done !
Moviepy - video ready /home/jay/Downloads/sat_tained/satellite_images.mp4


In [3]:
files= os.listdir('/home/jay/Downloads/sat_tained')
frames= []

for f in files:
    frames.append(os.path.join('/home/jay/Downloads/sat_tained', f))

In [6]:
!pwd

/home/jay/ipynb files
